In [1]:
import http.server
import threading
import numpy as np
import urllib.request
import OpenVisus as ov

Starting OpenVisus C:\Users\klacansky\AppData\Local\Programs\Python\Python310\lib\site-packages\OpenVisus\__init__.py 3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)] sys.version_info(major=3, minor=10, micro=8, releaselevel='final', serial=0) ...


In [2]:
urllib.request.urlretrieve('https://klacansky.com/open-scivis-datasets/hydrogen_atom/hydrogen_atom_128x128x128_uint8.raw', 'hydrogen_atom_128x128x128_uint8.raw')

('hydrogen_atom_128x128x128_uint8.raw',
 <http.client.HTTPMessage at 0x1ff75de6ec0>)

In [3]:
data = np.fromfile('hydrogen_atom_128x128x128_uint8.raw', dtype=np.uint8).reshape(128, 128, 128)

In [4]:
db = ov.CreateIdx(url="hydrogen_atom.idx", dims=list(reversed(data.shape)), fields=[ov.Field('data','uint8')], arco='1mb')
db.write(data)

In [5]:
def run():
    PORT = 8000
    Handler = http.server.SimpleHTTPRequestHandler
    with http.server.socketserver.TCPServer(("", PORT), Handler) as httpd:
        httpd.serve_forever()

threading.Thread(target=run).start()

In [6]:
db = ov.LoadDataset('http://localhost:8000/hydrogen_atom.idx')
data_ = db.read()

127.0.0.1 - - [26/Nov/2022 14:19:37] "GET /hydrogen_atom.idx HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 14:19:37] "GET /hydrogen_atom/0/data/0000/0000/0000/0001.bin HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 14:19:37] "GET /hydrogen_atom/0/data/0000/0000/0000/0000.bin HTTP/1.1" 200 -


In [7]:
assert (data == data_).all()